In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType

In [2]:
spark = SparkSession.builder.appName("Home Loan Approval System").getOrCreate()

In [3]:
application_data_schema = StructType([
    StructField("Customername",StringType(),True),
    StructField("DOB",StringType(),True),
    StructField("UIN",StringType(),True),
    StructField("MailID",StringType(),True),
    StructField("PhoneNumber",LongType(),True),
    StructField("City",StringType(),True),
    StructField("State",StringType(),True),
    StructField("LivingStatus",StringType(),True),
    StructField("PinCode",StringType(),True),
    StructField("LoanAmount",LongType(),True)
])

In [4]:
client_reference_schema = StructType([
    StructField("CustomerName",StringType(),True),
    StructField("DOB",StringType(),True),
    StructField("UIN",StringType(),True),
    StructField("City",StringType(),True),
    StructField("State",StringType(),True),
    StructField("PinCode",LongType(),True),
    StructField("CibilScore",LongType(),True),
    StructField("DefaulterFlag",StringType(),True)
])

In [5]:
application_data_df = spark.read.csv('./HomeLoanApplicationData.csv', schema= application_data_schema)

In [6]:
client_reference_df = spark.read.csv('./ClientReferenceDataset.csv', schema=client_reference_schema)

In [7]:
application_data_df.show(5)

+------------+--------+-------+-----------------+-----------+-----------------+-----------+------------+-------+----------+
|Customername|     DOB|    UIN|           MailID|PhoneNumber|             City|      State|LivingStatus|PinCode|LoanAmount|
+------------+--------+-------+-----------------+-----------+-----------------+-----------+------------+-------+----------+
|    Abhishek|23-08-81|UIN0001|Abhishek@mail.com| 1287655553|        Ahmedabad|    Gujarat|         APL| 380001|     23000|
|        Aman|17-09-84|UIN0002|    Aman@mail.com| 9776543221|           Mumbai|Maharashtra|         BPL| 400001|     22500|
|       Harsh|17-03-83|UIN0003|   Harsh@mail.com| 9987765234|Thiruvananthpuram|     Kerala|         BPL| 695001|     21500|
|       Ayush|22-02-78|UIN0004|   Ayush@mail.com| 4335544553|Thiruvananthpuram|     Kerala|         APL| 695001|     21500|
|       Aditi|21-08-79|UIN0005|   Aditi@mail.com| 8888777722|           Mumbai|Maharashtra|         BPL| 400001|     25500|
+-------

In [8]:
client_reference_df.show(5)

+------------+--------+-------+-----------------+-----------+-------+----------+-------------+
|CustomerName|     DOB|    UIN|             City|      State|PinCode|CibilScore|DefaulterFlag|
+------------+--------+-------+-----------------+-----------+-------+----------+-------------+
|    Abhishek|23-08-81|UIN0001|        Ahmedabad|    Gujarat| 380001|       910|            N|
|        Aman|17-09-84|UIN0002|           Mumbai|Maharashtra| 400001|       910|            N|
|       Harsh|17-03-83|UIN0003|Thiruvananthpuram|     Kerala| 695001|       530|            N|
|       Ayush|22-02-78|UIN0004|Thiruvananthpuram|     Kerala| 695001|       530|            N|
|       Aditi|21-08-79|UIN0005|           Mumbai|Maharashtra| 400001|      1300|            Y|
+------------+--------+-------+-----------------+-----------+-------+----------+-------------+
only showing top 5 rows



In [15]:
combined_df = application_data_df.join(other= client_reference_df, on='UIN').drop(client_reference_df.CustomerName).drop(client_reference_df.DOB).drop(client_reference_df.City).drop(client_reference_df.State).drop(client_reference_df.PinCode)

In [16]:
combined_df.show()

+-------+------------+--------+-----------------+-----------+-----------------+-----------+------------+-------+----------+----------+-------------+
|    UIN|Customername|     DOB|           MailID|PhoneNumber|             City|      State|LivingStatus|PinCode|LoanAmount|CibilScore|DefaulterFlag|
+-------+------------+--------+-----------------+-----------+-----------------+-----------+------------+-------+----------+----------+-------------+
|UIN0001|    Abhishek|23-08-81|Abhishek@mail.com| 1287655553|        Ahmedabad|    Gujarat|         APL| 380001|     23000|       910|            N|
|UIN0002|        Aman|17-09-84|    Aman@mail.com| 9776543221|           Mumbai|Maharashtra|         BPL| 400001|     22500|       910|            N|
|UIN0003|       Harsh|17-03-83|   Harsh@mail.com| 9987765234|Thiruvananthpuram|     Kerala|         BPL| 695001|     21500|       530|            N|
|UIN0004|       Ayush|22-02-78|   Ayush@mail.com| 4335544553|Thiruvananthpuram|     Kerala|         APL| 6

In [17]:
combined_df.createOrReplaceTempView('combined_table')

In [18]:
defaulters = spark.sql("Select * from combined_table where DefaulterFlag = 'Y' ")

In [19]:
defaulters.show()

+-------+------------+--------+-----------------+-----------+-----------------+-----------+------------+-------+----------+----------+-------------+
|    UIN|Customername|     DOB|           MailID|PhoneNumber|             City|      State|LivingStatus|PinCode|LoanAmount|CibilScore|DefaulterFlag|
+-------+------------+--------+-----------------+-----------+-----------------+-----------+------------+-------+----------+----------+-------------+
|UIN0005|       Aditi|21-08-79|   Aditi@mail.com| 8888777722|           Mumbai|Maharashtra|         BPL| 400001|     25500|      1300|            Y|
|UIN0008|     Anushka|25-08-82| Anushka@mail.com| 3344990876|Thiruvananthpuram|     Kerala|         APL| 695001|     24500|      1530|            Y|
|UIN0010|     Saurabh|22-08-83| Saurabh@mail.com| 3344990876|           Mumbai|Maharashtra|         APL| 400001|     20500|      4530|            Y|
|UIN0020|       Kajal|22-08-85|   Kajal@mail.com| 3344990876|           Mumbai|Maharashtra|         APL| 4

In [20]:
bpl = spark.sql("Select count(UIN) from combined_table where LivingStatus = 'BPL'")

In [21]:
bpl.show()

+----------+
|count(UIN)|
+----------+
|        28|
+----------+



In [22]:
apl = spark.sql("Select count(UIN) from combined_table where LivingStatus = 'APL'")

In [23]:
apl.show()

+----------+
|count(UIN)|
+----------+
|        32|
+----------+



In [29]:
status = spark.sql("select *, IF(DefaulterFlag == 'N' AND CibilScore > 800, 'Approved', 'Rejected') AS Status from combined_table")

In [30]:
status.show()

+-------+------------+--------+-----------------+-----------+-----------------+-----------+------------+-------+----------+----------+-------------+--------+
|    UIN|Customername|     DOB|           MailID|PhoneNumber|             City|      State|LivingStatus|PinCode|LoanAmount|CibilScore|DefaulterFlag|  Status|
+-------+------------+--------+-----------------+-----------+-----------------+-----------+------------+-------+----------+----------+-------------+--------+
|UIN0001|    Abhishek|23-08-81|Abhishek@mail.com| 1287655553|        Ahmedabad|    Gujarat|         APL| 380001|     23000|       910|            N|Approved|
|UIN0002|        Aman|17-09-84|    Aman@mail.com| 9776543221|           Mumbai|Maharashtra|         BPL| 400001|     22500|       910|            N|Approved|
|UIN0003|       Harsh|17-03-83|   Harsh@mail.com| 9987765234|Thiruvananthpuram|     Kerala|         BPL| 695001|     21500|       530|            N|Rejected|
|UIN0004|       Ayush|22-02-78|   Ayush@mail.com| 43

In [37]:
eligible = spark.sql("select * from combined_table where LivingStatus = 'BPL' AND DefaulterFlag = 'N' AND CibilScore > 800 ")

In [38]:
eligible.show()

+-------+------------+--------+----------------+-----------+-----------------+-----------+------------+-------+----------+----------+-------------+
|    UIN|Customername|     DOB|          MailID|PhoneNumber|             City|      State|LivingStatus|PinCode|LoanAmount|CibilScore|DefaulterFlag|
+-------+------------+--------+----------------+-----------+-----------------+-----------+------------+-------+----------+----------+-------------+
|UIN0002|        Aman|17-09-84|   Aman@mail.com| 9776543221|           Mumbai|Maharashtra|         BPL| 400001|     22500|       910|            N|
|UIN0007|     Shubham|23-08-86|Shubham@mail.com|44557768443|Thiruvananthpuram|     Kerala|         BPL| 695001|     21000|      3530|            N|
|UIN0009|       Rohit|23-08-83|  Rohit@mail.com| 3344990876|        Ahmedabad|    Gujarat|         BPL| 380001|     22000|      3530|            N|
|UIN0032|      Shreya|17-09-84| Shreya@mail.com| 9776543221|           Mumbai|Maharashtra|         BPL| 400001| 